In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2022-06-03 04:04:37--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   189MB/s    in 0.2s    

2022-06-03 04:04:37 (189 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-06-03 04:04:37--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [3]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [4]:
import json
with open('KorQuAD_v1.0_train.json') as f:
    data = json.load(f)

In [5]:
data

Output hidden; open in https://colab.research.google.com to view.

In [8]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)        # json 읽어들이고, squad dict변수에 저장
    
    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    return contexts, questions, answers

In [19]:
train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [20]:
print('훈련 데이터의 본문 개수 : ', len(train_contexts))
print('훈련 데이터의 질문 개수 : ', len(train_questions))
print('훈련 데이터의 답변 개수 : ', len(train_answers))
print('테스트 데이터의 본문 개수 : ', len(val_contexts))
print('테스트 데이터의 질문 개수 : ', len(val_questions))
print('테스트 데이터의 답변 개수 : ', len(val_answers))

훈련 데이터의 본문 개수 :  60407
훈련 데이터의 질문 개수 :  60407
훈련 데이터의 답변 개수 :  60407
테스트 데이터의 본문 개수 :  5774
테스트 데이터의 질문 개수 :  5774
테스트 데이터의 답변 개수 :  5774


In [14]:
print('첫 번째 샘플의 본문')
print('-------------------')
print(train_contexts[0])

첫 번째 샘플의 본문
-------------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [15]:
print('첫 번째 샘플의 질문')
print('-------------------')
print(train_questions[0])

첫 번째 샘플의 질문
-------------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [16]:
print('첫 번째 샘플의 답변')
print('-------------------')
print(train_answers[0])

첫 번째 샘플의 답변
-------------------
{'text': '교향곡', 'answer_start': 54}


In [17]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()                # 오른쪽 공백 제거
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text, 'end_index 계산에 에러가 있습니다.'           # 예외처리 assert 조건, 오류메시지  ->  assert 조건이 만족하지 않을 경우, 오류 메시지가 출력
        answer['answer_end'] = end_idx

In [21]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [22]:
print('첫 번째 샘플의 답변')
print('-------------------')
print(train_answers[0])

첫 번째 샘플의 답변
-------------------
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [23]:
train_contexts[0][54]

'교'

In [24]:
train_contexts[0][55]

'향'

In [25]:
train_contexts[0][55]

'향'

In [26]:
train_contexts[0][54:57]

'교향곡'

In [27]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

# 텍스트 -> 버트 서브워트토큰화 -> 정수 시퀀스 변환 -> 패딩
train_encodings = tokenizer(train_contexts, train_questions, truncation = True, padding = True)
val_encodings = tokenizer(val_contexts, val_questions, truncation = True, padding = True)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [29]:
train_encodings[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [79]:
# char_to_token : 원래의 문자열에서 토큰의 인덱스를 가져오는 역할
train_encodings.char_to_token(0, train_answers[0]['answer_start']) # (char_index, word index) return token index

26

In [80]:
train_encodings.token_to_chars(26)            # token index

CharSpan(start=54, end=57)

In [81]:
train_contexts[0][54:57]                     # start_position : end_position

'교향곡'

In [35]:
train_encodings[0].ids

[2,
 13934,
 2236,
 2440,
 27982,
 2259,
 21310,
 2079,
 11994,
 3791,
 2069,
 3790,
 1508,
 2088,
 636,
 3800,
 2170,
 3717,
 2052,
 9001,
 8345,
 4642,
 2200,
 3689,
 3657,
 2079,
 19282,
 2069,
 1363,
 2370,
 2259,
 936,
 2069,
 554,
 2259,
 2062,
 18,
 1504,
 4342,
 27982,
 2259,
 13934,
 2196,
 2440,
 2170,
 1195,
 23260,
 6233,
 17370,
 2113,
 2165,
 2069,
 809,
 1,
 3706,
 2052,
 2181,
 8642,
 2145,
 7334,
 2170,
 4983,
 2371,
 4007,
 1065,
 5917,
 2386,
 2559,
 4443,
 2138,
 4026,
 2259,
 11994,
 3791,
 2079,
 15864,
 2170,
 5487,
 2371,
 4683,
 3605,
 18,
 3819,
 5986,
 27135,
 1376,
 2645,
 2203,
 2292,
 2079,
 5872,
 2200,
 5986,
 4152,
 2252,
 22835,
 16706,
 2052,
 5485,
 2205,
 2259,
 17087,
 2079,
 19282,
 29,
 2517,
 2069,
 881,
 2088,
 652,
 2073,
 23404,
 2069,
 1122,
 2886,
 13964,
 16,
 3982,
 2052,
 9944,
 21,
 2429,
 2170,
 11994,
 3791,
 2079,
 1258,
 2465,
 6233,
 24294,
 1504,
 3967,
 2170,
 4027,
 2052,
 5121,
 3979,
 2069,
 18274,
 21575,
 23548,
 575,
 2073,

In [36]:
train_encodings[0].offsets

[(0, 0),
 (0, 3),
 (3, 4),
 (4, 5),
 (6, 9),
 (9, 10),
 (11, 13),
 (13, 14),
 (15, 17),
 (17, 19),
 (19, 20),
 (21, 23),
 (24, 25),
 (25, 26),
 (27, 28),
 (29, 31),
 (31, 32),
 (33, 35),
 (35, 36),
 (37, 39),
 (40, 42),
 (43, 45),
 (45, 46),
 (47, 49),
 (50, 52),
 (52, 53),
 (54, 57),
 (57, 58),
 (59, 60),
 (60, 61),
 (61, 62),
 (63, 64),
 (64, 65),
 (66, 67),
 (67, 68),
 (68, 69),
 (69, 70),
 (71, 72),
 (73, 75),
 (76, 79),
 (79, 80),
 (81, 84),
 (84, 85),
 (85, 86),
 (86, 87),
 (88, 89),
 (90, 92),
 (92, 94),
 (95, 97),
 (97, 98),
 (98, 99),
 (99, 100),
 (101, 102),
 (103, 105),
 (106, 108),
 (108, 109),
 (109, 110),
 (111, 113),
 (113, 114),
 (115, 117),
 (117, 118),
 (119, 121),
 (121, 122),
 (122, 124),
 (125, 126),
 (126, 128),
 (128, 129),
 (129, 130),
 (130, 132),
 (132, 133),
 (134, 136),
 (136, 137),
 (138, 140),
 (140, 142),
 (142, 143),
 (144, 146),
 (146, 147),
 (148, 150),
 (150, 151),
 (151, 153),
 (154, 156),
 (156, 157),
 (158, 160),
 (161, 163),
 (163, 165),
 (166, 16

In [41]:
# 음절 단위로 읽는거지만, bert tokenizer는 wordpiece 단위
# 그래서 음절 단위에 있는 숫자를 tokeniz index로 바꿔줘야함
# 그래야지 해당 token index가 정답 label의 시작이다 라는 것을 알 수 있기 때문에 

def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    # 음절 index를 token index와 mapping하는 작업 중
    for i in tqdm(range(len(answers))):

        # tokenizer의 char_to_token 함수를 호출하면, 음절 숫자를 token index로 바꿔줄 수 있다.
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # truncation을 위한 작업
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
                deleting_list.append(i)
    encodings.update({'start_positions' : start_positions, 'end_positions' : end_positions})
    return deleting_list

In [42]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 104669.07it/s]


In [45]:
def delete_samples(encodings, deleting_list):
    input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis= 0)
    attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis= 0)
    start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis= 0)
    end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis= 0)

    # 해당 문장의 ids, attention_mask를 concat해서 x_data를 완성
    x_data = [input_ids, attention_masks]
    
    # 정답의 첫번째 위치, 마지막 위치 concat해서 y_data를 완성
    y_data = [start_positions, end_positions]

    return x_data, y_data

In [47]:
x_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
x_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [48]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.5.102.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.5.102.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [64]:
from transformers import TFBertModel

In [67]:
class TFBertQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                               kernel_initializer = tf.keras.initializers.TruncatedNormal(0.02),
                                               name = 'qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask = attention_mask)            # -> Dense 층을 2로 잡았기 때문에 2개의 결과가 나온다

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)         # Dense 층 통과
        start_logits, end_logits = tf.split(logits, 2, axis = -1)

        # start_logits = (batch_size, sequence_length, ) -> 쓸모없는 차원이 증가한 상태
        # end_logits = (batch_size, sequence_length, ) -> 쓸모없는 차원이 증가한 상태
        start_logits = tf.squeeze(start_logits, axis = -1)
        end_logits = tf.squeeze(end_logits, axis = -1)                        # squeeze : 차원을 줄임

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)               # 확률 구하기

        return start_probs, end_probs

In [68]:
with strategy.scope():
    model = TFBertQuestionAnswering("klue/bert-base")
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=optimizer, loss=loss)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [69]:
history = model.fit(
    x_train,
    y_train,
    epochs = 5,
    verbose = 1,
    batch_size = 16
)

Epoch 1/5


3026/3026 [==============================] - 433s 113ms/step - loss: 1.0485 - output_1_loss: 0.5256 - output_2_loss: 0.5228
Epoch 2/5
3026/3026 [==============================] - 344s 114ms/step - loss: 0.5166 - output_1_loss: 0.2591 - output_2_loss: 0.2575
Epoch 3/5
3026/3026 [==============================] - 344s 114ms/step - loss: 0.3574 - output_1_loss: 0.1789 - output_2_loss: 0.1785
Epoch 4/5
3026/3026 [==============================] - 345s 114ms/step - loss: 0.2818 - output_1_loss: 0.1400 - output_2_loss: 0.1418
Epoch 5/5
3026/3026 [==============================] - 344s 114ms/step - loss: 0.2409 - output_1_loss: 0.1209 - output_2_loss: 0.1200


In [77]:
def predict_test_data_by_idx(idx):
    context = tokenizer.decode(x_test[0][idx]).split('[SEP] ')[0]
    question = tokenizer.decode(x_test[0][idx]).split('[SEP] ')[1]
    print('본문 : ', context)
    print('질문 : ', question)

    # x_test[0] : input_ids
    # x_test[1] : attention_mask
    # x_test[0]'[0] : input_ids에서 첫 번째 문장
    # x_test[0][0][y_test[0][idx] : y_test[1][idx] + 1] : 토큰 인덱스 접근 (첫 위치 : 끝 위치 )
    answer_encoded = x_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
    print('정답 : ', tokenizer.decode(answer_encoded))

    output = model([tf.constant(x_test[0][idx])[None, :], tf.constant(x_test[1][idx])[None, :]])
    start = tf.math.argmax(tf.squeeze(output[0]))
    end = tf.math.argmax(tf.squeeze(output[1])) +1
    answer_encoded = x_test[0][idx][start:end]
    print('예측 : ', tokenizer.decode(answer_encoded))
    print('-----------------------------------------')

In [78]:
for i in range(0, 100):
    predict_test_data_by_idx(i)

본문 :  [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 :  1989년 6월 30일 평양축전에 대표로 파견 된 인물은? 
정답 :  임수경을
예측 :  임수경을
-----------------------------------------
본문 :  [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는